# LLMCheckerChain
This notebook showcases how to use LLMCheckerChain.

In [1]:
from langchain.chains import LLMCheckerChain, LLMChain, SequentialChain, ParallelChain
from langchain.llms import OpenAI
from langchain.prompts.base import ListOutputParser
from langchain.llms.base import LLM


llm = OpenAI(temperature=0.7)

text = "What type of mammal lays the biggest eggs?"

checker_chain = LLMCheckerChain(llm=llm, verbose=True)

# checker_chain.run(text)

In [2]:
# flake8: noqa
from langchain.prompts.prompt import PromptTemplate

_CREATE_DRAFT_ANSWER_TEMPLATE = """Question: {question}

Answer: Let's think step by step. """
CREATE_DRAFT_ANSWER_PROMPT = PromptTemplate(
    input_variables=["question"], template=_CREATE_DRAFT_ANSWER_TEMPLATE
)

_LIST_ASSERTIONS_TEMPLATE = """{statement}
Make a bullet point list of the assumptions you made when producing the above statement.\n\n"""
LIST_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["statement"], template=_LIST_ASSERTIONS_TEMPLATE
)

_CHECK_ASSERTIONS_TEMPLATE = """{assertions}
Repeat the assertion above, and then determine if it is true or false. If it is false, explain why.\n\n"""
CHECK_ASSERTIONS_PROMPT = PromptTemplate(
    input_variables=["assertions"], template=_CHECK_ASSERTIONS_TEMPLATE
)

_REVISED_ANSWER_TEMPLATE = """{checked_assertions}

Question: In light of the above assertions and checks, how would you answer the question '{question}'?

The answer is: """
REVISED_ANSWER_PROMPT = PromptTemplate(
    input_variables=["checked_assertions", "question"],
    template=_REVISED_ANSWER_TEMPLATE,
)

In [3]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement"
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
    output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n".join(x),
)

self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["revised_answer"],
    verbose=True,
)

In [4]:
# %%time
# self_ask_verification_chain.run(text)

In [38]:
llm = OpenAI(temperature=0.7)

create_draft_answer_chain = ParallelChain(
    llm=llm, prompt=CREATE_DRAFT_ANSWER_PROMPT, output_key="statement", n = 25
)

list_assertions_chain = ParallelChain(
    llm=llm,
    prompt=LIST_ASSERTIONS_PROMPT,
    output_key="assertions",
    # This indicates that there are multiple outputs
#     output_parser=lambda x: [i.strip() for i in x.split("\n") if len(i.strip()) > 0],
)

check_assertions_chain = ParallelChain(
    llm=llm, prompt=CHECK_ASSERTIONS_PROMPT, output_key="checked_assertions"
)

revised_answer_chain = ParallelChain(
    llm=llm,
    prompt=REVISED_ANSWER_PROMPT,
    output_key="revised_answer",
    # This indicates that there are multiple inputs
#     input_parser=lambda x: "\n".join(x),
)


MOE_CONSENSUS_PROMPT = PromptTemplate(
    input_variables=["revised_answer", "question"],
    template="""{revised_answer}

A group of people each gave their answers above. Write an answer to '{question}' that would be the least controversial to this group.

Answer: """,
)

moe_answer_chain = ParallelChain(
    llm=llm,
    prompt=MOE_CONSENSUS_PROMPT,
    output_key="final_answer",
    # This indicates that there are multiple inputs
    input_parser=lambda x: "\n -" + "\n -".join(x),
)


self_ask_verification_chain = SequentialChain(
    chains=[
        create_draft_answer_chain,
        list_assertions_chain,
        check_assertions_chain,
        revised_answer_chain,
        moe_answer_chain,
    ],
    input_variables=["question"],
    output_variables=["final_answer"],
    verbose=True,
)

In [44]:
%%time
self_ask_verification_chain.run("What type of mammal lays the biggest eggs?")



> Entering new SequentialChain chain...
Chain 0:
{'statement': [' Mammals are animals that give birth to live young and feed them milk from mammary glands. They do not lay eggs. So, the answer to this question is that no type of mammal lays eggs.', '\n\n1. What type of mammal lays eggs?\n\nAnswer: Monotremes, which are mammals that lay eggs, are the only mammals that lay eggs.\n\n2. What monotreme lays the biggest eggs?\n\nAnswer: The largest monotreme egg is laid by the echidna, which can lay eggs up to 10 cm in length.', "\n\nFirst, let's consider the mammals. Mammals are warm-blooded vertebrates that have hair or fur, feed their young with milk produced by mammary glands, and have a more complex nervous system than other animals. \n\nNow, let's consider eggs. Egg-laying mammals, or monotremes, lay eggs instead of giving birth to live young. The three species of monotremes are the platypus, the echidna, and the two species of spiny anteaters. \n\nOf these, the echidna lays the bigg

Chain 2:
{'checked_assertions': ['Mammals lay eggs.\nFalse. Mammals do not lay eggs; they give birth to live young.', 'The assertion is: Echidnas lay the largest monotreme eggs.\n\nThis statement is true. Echidnas lay the largest monotreme eggs, which can be up to 10 cm in length.', 'The assertion is that the echidna lays the biggest eggs among the monotremes. This is true. Echidna eggs weigh around 10 grams and are about 25 mm long, which is larger than the eggs laid by the platypus and the two species of spiny anteaters.', '\nAssertion: Mammals lay eggs.\nFalse. Mammals typically give birth to live young and no mammal lays eggs.', 'The assertion: All animals that lay eggs are not mammals. \n\nThis is true. Mammals give birth to live young, and not lay eggs.', 'The assertion: No mammal lays eggs.\n\nThis assertion is true. Mammals give birth to live offspring, rather than laying eggs, which is a trait shared by birds, reptiles, and amphibians.', 'The assertion: Mammals are warm-bloode

[' Monotremes lay the biggest eggs of any mammal.']

In [45]:
%%time
self_ask_verification_chain.run("Where did Mark Zuckerberg first meet Sheryl Sandberg?")



> Entering new SequentialChain chain...
Chain 0:
{'statement': ['\n\nStep 1: Find out when Mark Zuckerberg and Sheryl Sandberg first met. \n\nMark Zuckerberg and Sheryl Sandberg first met in 2007. \n\nStep 2: Find out where they met. \n\nThey met at a holiday party hosted by venture capitalist Jim Breyer in 2007.', '\n\nStep 1: Do a quick online search for "Mark Zuckerberg + Sheryl Sandberg". \n\nStep 2: Review the results of the search. \n\nStep 3: Based on the search results, it appears that Mark Zuckerberg and Sheryl Sandberg first met at a Christmas party in 2007.', "\n\nFirst, let's look at what we know. We know that Mark Zuckerberg is the founder and CEO of Facebook, and that Sheryl Sandberg is the chief operating officer of Facebook. \n\nNext, let's do a bit of research. According to an article by The New York Times, Mark Zuckerberg and Sheryl Sandberg first met in 2007 at a Christmas party hosted by a mutual friend.", '\n\nStep 1: Research. \n\nStep 2: According to reports, M

Chain 2:
{'checked_assertions': ['\nMark Zuckerberg and Sheryl Sandberg met in person at a holiday party hosted by Jim Breyer in 2007. True', '\nAssertion: Mark Zuckerberg and Sheryl Sandberg first met at a Christmas party in 2007.\n\nAnswer: True', 'Assertion: Mark Zuckerberg and Sheryl Sandberg met in 2007 at a Christmas party hosted by a mutual friend.\n\nTrue. This is accurate information as Mark Zuckerberg and Sheryl Sandberg met in 2007 at a Christmas party hosted by a mutual friend.', '\nThe assertion: Mark Zuckerberg and Sheryl Sandberg met each other at the Christmas party held at the home of Jim Breyer in December 2007.\n\nThis assertion is true. Mark Zuckerberg and Sheryl Sandberg both attended the same Christmas party held at the home of Jim Breyer in December 2007, and it was at the party that they first met.', 'Assertion: Mark Zuckerberg and Sheryl Sandberg met before 2006.\nFalse - There is no evidence to suggest that Mark Zuckerberg and Sheryl Sandberg met before 2006. 

[' Mark Zuckerberg and Sheryl Sandberg first met in 2007 at a holiday party hosted by a mutual friend.']

In [49]:
%%time
self_ask_verification_chain.run("Where can I buy beer if it's late in NYC and I'm in the flatiron area?")



> Entering new SequentialChain chain...
Chain 0:
{'statement': ["\n\nFirst, you need to check the local laws in New York City to make sure that you can purchase beer at the time you are looking for. In New York City, the sale of beer is allowed until 4am.\n\nNext, you need to locate a store nearby. There are a few convenience stores and bodegas in the Flatiron area that are open late and sell beer. You can also visit a liquor store if you're looking for a wider selection. Some of the popular liquor stores in the area are Pogo's Wine and Liquors, Dave's Wine and Liquors, and Astor Wines.", "\nFirst, you'll want to make sure you check the local laws. In New York City, beer can be sold until 4:00am in bars, restaurants, and convenience stores. \nNext, you'll need to find a place that is open late. Some convenience stores in the Flatiron area that may be open late include 7-Eleven on Park Avenue South, Rite Aid on Broadway, and Duane Reade on Fifth Avenue. You could also try a local bar 

Chain 1:
{'assertions': ['\n• Assumed that the person looking to purchase beer is of legal drinking age.\n• Assumed that the person is looking to purchase beer in the Flatiron area of New York City.\n• Assumed that the person is looking to purchase beer late at night or in the early morning.\n• Assumed that the local laws in New York City allow for the sale of beer until 4am.', '\n-Local laws in New York City allow beer to be sold until 4:00am in bars, restaurants, and convenience stores \n-Convenience stores in the Flatiron area may be open late\n-Local bars and restaurants may be open late\n-Valid form of identification is required to purchase beer in New York City \n-Must be 21 years old or older to purchase beer', '\n• Assumed the store was in the Flatiron area \n• Assumed the store was a convenience store \n• Assumed the store was open late \n• Checked 7-11 and Deli Grocery \n• Checked Flatiron Wine & Spirits as an alternative', '\n• Alcohol can be legally purchased in the area\n•

Chain 2:
{'checked_assertions': ['Assertion: The local laws in New York City allow for the sale of beer until 4am.\n\nFalse. The New York State Alcoholic Beverage Control Law prohibits the sale of alcoholic beverages between the hours of 4 a.m. and 8 a.m. in New York City.', 'The assertion that local laws in New York City allow beer to be sold until 4:00am in convenience stores is false. While beer can be sold until 4:00am in bars and restaurants, convenience stores are only allowed to sell beer until 2:00am in New York City.', '\nAssertion: The store is open late.\n\nTrue. 7-11 and Deli Grocery are both open late in the Flatiron area. Flatiron Wine & Spirits is also open late, though it is not a convenience store.', 'The assertion is: Alcohol can be legally purchased in the area. \nThis is true, as long as the individual purchasing the alcohol is of legal age and the vendor is licensed to sell alcohol.', 'Assertion: Convenience stores in the Flatiron area offer a selection of domestic

Chain 4:
{'final_answer': ['\nYou can purchase beer late at night in the Flatiron area in NYC from convenience stores, delis, bodegas, and delivery services. However, you must be of legal drinking age and provide valid identification to purchase the beer.']}


> Finished SequentialChain chain.
CPU times: user 1.66 s, sys: 227 ms, total: 1.89 s
Wall time: 18.5 s


['\nYou can purchase beer late at night in the Flatiron area in NYC from convenience stores, delis, bodegas, and delivery services. However, you must be of legal drinking age and provide valid identification to purchase the beer.']

In [ ]:
%%time
self_ask_verification_chain.run("Where can I buy beer if it's late in NYC and I'm in the flatiron area?")

In [47]:
%%time
question = """Question: Where can I buy beer if it's late in NYC and I'm in the flatiron area?

Answer: Let's think step by step."""
results = llm(question, best_of=25, n = 5)

CPU times: user 37.5 ms, sys: 4.74 ms, total: 42.2 ms
Wall time: 8.12 s


In [48]:
print(sum([int('echidna' in r.lower()) for r in results]))
moe_answer_chain({'revised_answer': results, 'question': question})

0


{'revised_answer': [' First, you\'ll want to check if any local liquor stores are open late. You can search for "late night liquor stores near Flatiron, NYC" to find out. If you can\'t find any, you can also check for any nearby bars or restaurants that serve beer. You can search for "bars near Flatiron, NYC" or "restaurants near Flatiron, NYC" to find out.',
  " \n\nFirst, you'll need to check if there are any convenience stores near you that sell beer. You can try searching on Google Maps or Yelp for nearby convenience stores. \n\nIf you can't find any convenience stores, then you can look for liquor stores that are open late. Again, you can search for these on Google Maps or Yelp. \n\nIf you can't find any liquor stores, then you can try searching for bars and pubs that are open late. These establishments typically serve beer, so you should be able to find one near the flatiron area.",
  " First, New York State law requires that beer be purchased in a liquor store, so you'll need to

In [10]:
# %%time
# self_ask_verification_chain.run("Where did Sheryl Sandberg meet Mark Zuckerberg?")